In [1]:
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from pathlib import Path

In [4]:
path = Path("piano_music/")

In [5]:
from data import AudioMP3Dataset, collate_fn

In [6]:
dataset = AudioMP3Dataset(path)

In [7]:
from model import WaveNet

In [8]:
res_layers = 8
res_blocks = 2
casual_layers = 1
kernal_size = 7
quantize_channels = 256

model = WaveNet(res_layers, res_blocks, casual_layers, kernal_size=kernal_size)

In [9]:
receptive_field = model.receptive_field
receptive_field

3061

In [10]:
from torch.utils.data import DataLoader

In [11]:
from functools import partial

In [12]:
max_len = 100000
collate_fn_max = partial(collate_fn, max_len=max_len)

In [13]:
dl = DataLoader(dataset, shuffle=True, batch_size=1, collate_fn=collate_fn_max)

In [14]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()
model = model.to(device)

In [15]:
epochs = 10
checkpoint_path = Path("model_checkpoints/")
saved_states = 0

In [16]:
losses = []
for i in range(epochs):
    for inputs in dl:
        inputs = inputs.to(device)
        for i in range(receptive_field+2, inputs.size(-1)-3):
            loss = model.oneMaskStep(inputs, i, optimizer, loss_fn, verbose=False)
        losses.append(loss.item())
        print(f"Loss: {loss.item()}")
    model.saveState(checkpoint_path/f"{saved_states}_checkpoint.pt")
    saved_states+=1

Loss: 0.017159122973680496
Loss: 0.016546493396162987
Loss: 0.017324890941381454
Loss: 0.017024600878357887
Loss: 0.018003104254603386
Loss: 0.01942410133779049


KeyboardInterrupt: 